In [1]:
import torch
import torch.nn as nn
import numpy as np
import os
from collections import deque

## 1D-CNN 特征提取器

In [ ]:
def create_raw_data_cnn():
    """创建一个用于处理原始传感器数据的1D-CNN模块。"""
    # 以下序贯模型来自 feature_edge_node_stimulate.ipynb 中 feature_extractor 的结构
    raw_data_processor = nn.Sequential(
        nn.Conv1d(in_channels=11, out_channels=64, kernel_size=3, padding='same'), 
        nn.ReLU(), 
        nn.BatchNorm1d(64),
        nn.MaxPool1d(kernel_size=2, stride=2), # Length: 200 -> 100
        
        nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding='same'), 
        nn.ReLU(), 
        nn.BatchNorm1d(128),
        nn.MaxPool1d(kernel_size=2, stride=2), # Length: 100 -> 50

        nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding='same'), 
        nn.ReLU(), 
        nn.BatchNorm1d(256),
        nn.MaxPool1d(kernel_size=2, stride=2), # Length: 50 -> 25
        
        nn.Flatten() # 输出一个扁平化的向量
    )
    return raw_data_processor

## 历史特征池管理器

In [ ]:
class HistoricalFeaturePool:
    """管理从边缘节点接收的历史特征，并提供固定长度的序列。"""
    def __init__(self, max_sequence_length=60):
        """
        初始化特征池。
        :param max_sequence_length: LSTM期望的序列长度。例如，60代表30秒的历史（每0.5秒一个特征）。
        """
        self.max_len = max_sequence_length
        # 使用deque可以高效地维持一个固定大小的队列
        self.feature_deque = deque(maxlen=self.max_len)
        print(f"Historical Feature Pool initialized with max length {self.max_len}.")

    def add_feature(self, feature_vector: np.ndarray):
        """添加一个新的特征向量到池中。"""
        if not isinstance(feature_vector, np.ndarray):
            raise TypeError("feature_vector must be a numpy array.")
        self.feature_deque.append(feature_vector)

    def get_feature_sequence(self):
        """
        获取当前的特征序列，用于输入到模型。
        如果特征不足，会用零向量在左侧填充。
        """
        current_sequence = list(self.feature_deque)
        current_len = len(current_sequence)
        
        if current_len == 0:
            # 获取第一个特征的维度
            first_feature_dim = 6400  # 根据 feature_edge_node_stimulate.ipynb，扁平化后是 256 * 25 = 6400
            return np.zeros((self.max_len, first_feature_dim))

        # 如果序列未满，进行左侧填充
        if current_len < self.max_len:
            padding_len = self.max_len - current_len
            feature_dim = current_sequence[0].shape[-1]
            padding = np.zeros((padding_len, feature_dim))
            return np.vstack([padding] + current_sequence)
        
        return np.array(current_sequence)

## 保真模型 - 基于门控机制

In [ ]:
class FidelityModelGated(nn.Module):
    def __init__(self, feature_dim, lstm_hidden_dim, raw_cnn_output_dim, num_classes=1):
        super(FidelityModelGated, self).__init__()
        self.lstm_hidden_dim = lstm_hidden_dim

        # 将 raw_cnn_output_dim 保存为实例属性
        self.raw_cnn_output_dim = raw_cnn_output_dim

        # 分支一：处理来自边缘节点的历史特征序列
        self.feature_lstm = nn.LSTM(
            input_size=feature_dim,
            hidden_size=lstm_hidden_dim,
            num_layers=2, # 使用两层LSTM增加模型深度
            batch_first=True, # 输入数据格式为 (batch, seq_len, feature_dim)
            dropout=0.5
        )

        # 分支二：处理高保真的原始传感器数据
        self.raw_data_cnn = create_raw_data_cnn()

        # 门控融合单元
        # 输入维度是LSTM输出和CNN输出的拼接
        self.gating_layer = nn.Sequential(
            nn.Linear(lstm_hidden_dim + raw_cnn_output_dim, lstm_hidden_dim),
            nn.ReLU(),
            nn.Linear(lstm_hidden_dim, 1),
            nn.Sigmoid()
        )

        # 将用于维度匹配的 raw_transform 层在 __init__ 中预先定义好
        self.raw_transform = nn.Linear(self.raw_cnn_output_dim, self.lstm_hidden_dim)

        # 最终分类器
        self.classifier = nn.Sequential(
            nn.Linear(lstm_hidden_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, num_classes)
        )

    def forward(self, feature_sequence, raw_data=None):
        """
        模型的前向传播。
        :param feature_sequence: (batch, seq_len, feature_dim) 的历史特征序列
        :param raw_data: (batch, 200, 11) 的原始传感器数据，或为 None
        :return: 模型的输出logits, LSTM的最后一个隐藏状态 (用于采样模型)
        """
        # 1. 处理历史特征序列
        lstm_outputs, (h_n, c_n) = self.feature_lstm(feature_sequence)
        
        # 只关心最后一个时间步的LSTM输出
        lstm_last_output = lstm_outputs[:, -1, :] # (batch, hidden_dim)

        # 2. 处理原始传感器数据 (如果可用)
        if raw_data is not None:
            # Conv1D需要 (batch, channels, length) 格式
            raw_data = raw_data.permute(0, 2, 1)
            v_raw = self.raw_data_cnn(raw_data) # (batch, raw_cnn_output_dim)
        else:
            # 使用预先保存的 self.raw_cnn_output_dim 创建零向量
            v_raw = torch.zeros(feature_sequence.size(0), self.raw_cnn_output_dim, device=feature_sequence.device)

        # 3. 门控融合
        # 计算门控值 g
        combined_for_gate = torch.cat((lstm_last_output, v_raw), dim=1)
        gate = self.gating_layer(combined_for_gate)
        
        # 直接使用在 __init__ 中定义好的 self.raw_transform 层
        transformed_v_raw = self.raw_transform(v_raw)

        fused_vector = lstm_last_output + gate * torch.tanh(transformed_v_raw)
        
        # 4. 分类
        logits = self.classifier(fused_vector)

        # 5. 返回结果和状态特征
        # h_n 的形状是 (num_layers, batch, hidden_dim), 取最后一层的状态
        state_feature = h_n[-1, :, :].squeeze(0) # (batch, hidden_dim)
        
        return logits, state_feature, gate.item()


## 创建历史数据（从数据库中抽取数据）（暂未完成）

## 模拟主流程

In [ ]:
# --- 配置 ---
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
FEATURE_DIR = "simulated_features" # 从 feature_edge_node_stimulate.ipynb 生成的特征文件夹

# 模型超参数
# 来自 notebook: 256 (channels) * 25 (length)
FEATURE_DIM = 256 * 25 
LSTM_HIDDEN_DIM = 256
# 来自 create_raw_data_cnn(): 256 (channels) * 25 (length)
RAW_CNN_OUTPUT_DIM = 256 * 25 
HISTORY_SEQ_LEN = 60 # 使用过去30秒的历史特征 (60个特征点)

# --- 初始化 ---
print("Initializing Fidelity Model and Feature Pool...")
fidelity_model = FidelityModelGated(
    feature_dim=FEATURE_DIM,
    lstm_hidden_dim=LSTM_HIDDEN_DIM,
    raw_cnn_output_dim=RAW_CNN_OUTPUT_DIM
).to(DEVICE)

# --- 从 .pth 文件加载模型权重 ---
# 定义模型权重文件路径
model_path = 'fidelity_model_best.pth'

# 加载状态字典 (state_dict)
# 注意：map_location 参数很重要，它可以确保无论模型是在 GPU 还是 CPU 上训练的，都能正确加载。
print(f"Loading trained model weights from {model_path}...")
state_dict = torch.load(model_path, map_location=DEVICE)

# 将加载的权重应用到模型实例上
fidelity_model.load_state_dict(state_dict)
print("Model weights loaded successfully.")

fidelity_model.eval() # 设为评估模式，确保在推理时关闭 Dropout 和 Batch Normalization 的训练行为

feature_pool = HistoricalFeaturePool(max_sequence_length=HISTORY_SEQ_LEN)

# --- 加载从边缘节点模拟器生成的特征 ---
print(f"Loading features from '{FEATURE_DIR}'...")
if not os.path.exists(FEATURE_DIR):
    raise FileNotFoundError(f"Feature directory '{FEATURE_DIR}' not found. Please run the notebook first.")

feature_files = sorted([f for f in os.listdir(FEATURE_DIR) if f.endswith('.npy')])
all_features = [np.load(os.path.join(FEATURE_DIR, f)) for f in feature_files]
edge_feature_stream = np.vstack(all_features)
print(f"Loaded a total of {len(edge_feature_stream)} feature vectors.")

# --- 【待办】模拟从数据库加载部分历史原始数据 ---
# 在真实场景中，应该从数据库查询。这里创建一个虚拟数据
# 假设在第100和第250个时间步从边缘节点同步了原始数据
print("Simulating sparse historical raw data...")
dummy_raw_data_snippet = np.random.rand(200, 11) # 形状 (200, 11)
raw_data_availability = {
    100: dummy_raw_data_snippet,
    250: dummy_raw_data_snippet * 0.5 # 用不同数据模拟
}

# --- 主模拟循环 ---
print("\n--- Starting Fidelity Model Simulation ---\n")
for i, feature_vector in enumerate(edge_feature_stream):
    
    #【网络接口】模拟接收到新特征并添加到池中
    feature_pool.add_feature(feature_vector)
    
    # 从池中获取当前的历史序列
    current_feature_sequence = feature_pool.get_feature_sequence()
    
    # 检查当前时间步是否有可用的原始数据
    current_raw_data = raw_data_availability.get(i, None)
    
    # 准备输入张量
    seq_tensor = torch.tensor(current_feature_sequence, dtype=torch.float32).unsqueeze(0).to(DEVICE)
    raw_tensor = torch.tensor(current_raw_data, dtype=torch.float32).unsqueeze(0).to(DEVICE) if current_raw_data is not None else None
    
    # 模型推理
    with torch.no_grad():
        logits, state_feature, gate_value = fidelity_model(seq_tensor, raw_tensor)

    # 解析和打印结果
    confidence = torch.sigmoid(logits).item()
    prediction = "FALL DETECTED!" if confidence > 0.5 else "No Fall"
    
    raw_data_info = "Yes" if current_raw_data is not None else "No"
    
    print(
        f"Step: {i+1:03d} | "
        f"Raw Data Used: {raw_data_info:<3} | "
        f"Gate Value: {gate_value:.2f} | "
        f"Confidence: {confidence:.4f} | "
        f"Prediction: {prediction:<15} | "
        f"State Feature Shape: {state_feature.shape}"
    )

Initializing Fidelity Model and Feature Pool...
Loading trained model weights from fidelity_model_best.pth...
Model weights loaded successfully.
Historical Feature Pool initialized with max length 60.
Loading features from 'simulated_features'...
Loaded a total of 380 feature vectors.
Simulating sparse historical raw data...

--- Starting Fidelity Model Simulation ---

Step: 001 | Raw Data Used: No  | Gate Value: 0.38 | Confidence: 0.9674 | Prediction: FALL DETECTED!  | State Feature Shape: torch.Size([256])
Step: 002 | Raw Data Used: No  | Gate Value: 0.29 | Confidence: 0.9984 | Prediction: FALL DETECTED!  | State Feature Shape: torch.Size([256])
Step: 003 | Raw Data Used: No  | Gate Value: 0.27 | Confidence: 0.9996 | Prediction: FALL DETECTED!  | State Feature Shape: torch.Size([256])
Step: 004 | Raw Data Used: No  | Gate Value: 0.28 | Confidence: 0.9996 | Prediction: FALL DETECTED!  | State Feature Shape: torch.Size([256])
Step: 005 | Raw Data Used: No  | Gate Value: 0.31 | Confiden

## 读懂结果

* **`Raw Data Used`**: 显示当前推理步骤是否利用了高保真的原始数据。
* **`Gate Value`**: 这是关键！当 `Raw Data Used` 为 `Yes` 时，如果这个值很低（例如\<0.5），说明模型判断新来的原始数据非常重要，更多地采纳了它。如果这个值很高（例如\>0.5），说明模型更相信自己基于历史特征的判断。当 `Raw Data Used` 为 `No` 时，这个值通常在0.5附近，因为全零的`v_raw`对门控影响不大。
* **`Prediction`**: 最终的判断结果。
* **`State Feature Shape`**: 这就是您要提供给采样模型的“状态特征”。它的形状是 `(1, 256)`（去掉了batch维度后是256），您可以直接将其传递给RL模型。

